# OpenAI APIs - Completions

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial covers the following popular APIs:

- `chat/completions`
- `completions`
- `batches`

Check out other tutorials to learn about [vision APIs](https://docs.sglang.ai/backend/openai_api_vision.html) for vision-language models and [embedding APIs](https://docs.sglang.ai/backend/openai_api_embeddings.html) for embedding models.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-18 06:24:43] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=39004, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=422742894, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, lo

The following error message 'operation scheduled before its operands' can be ignored.
The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-18 06:25:03 TP0] Init torch distributed begin.
[2025-03-18 06:25:03 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-03-18 06:25:03 TP0] Load weight begin. avail mem=78.81 GB


[2025-03-18 06:25:04 TP0] Using model weights format ['*.safetensors']


...

[2025-03-18 06:25:09 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-03-18 06:25:10] INFO:     Started server process [2850231]
[2025-03-18 06:25:10] INFO:     Waiting for application startup.
[2025-03-18 06:25:10] INFO:     Application startup complete.
[2025-03-18 06:25:10] INFO:     Uvicorn running on http://0.0.0.0:39004 (Press CTRL+C to quit)


[2025-03-18 06:25:11] INFO:     127.0.0.1:38492 - "GET /v1/models HTTP/1.1" 200 OK
[2025-03-18 06:25:11] INFO:     127.0.0.1:38508 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-18 06:25:11 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-18 06:25:13] INFO:     127.0.0.1:38524 - "POST /generate HTTP/1.1" 200 OK
[2025-03-18 06:25:13] The server is fired up and ready to roll!


Server started on http://localhost:39004


## Chat Completions

### Usage

The server fully implements the OpenAI API.
It will automatically apply the chat template specified in the Hugging Face tokenizer, if one is available.
You can also specify a custom chat template with `--chat-template` when launching the server.

In [2]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-03-18 06:25:16 TP0] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-18 06:25:16 TP0] Decode batch. #running-req: 1, #token: 76, token usage: 0.00, gen throughput (token/s): 5.55, #queue-req: 0, 
[2025-03-18 06:25:16] INFO:     127.0.0.1:38536 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

Here is an example of a detailed chat completion request:

In [3]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=128,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(response.choices[0].message.content)

[2025-03-18 06:25:16 TP0] Prefill batch. #new-seq: 1, #new-token: 51, #cached-token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-18 06:25:17 TP0] Decode batch. #running-req: 1, #token: 106, token usage: 0.01, gen throughput (token/s): 56.12, #queue-req: 0, 


[2025-03-18 06:25:18 TP0] Decode batch. #running-req: 1, #token: 146, token usage: 0.01, gen throughput (token/s): 52.91, #queue-req: 0, 


[2025-03-18 06:25:18 TP0] Decode batch. #running-req: 1, #token: 186, token usage: 0.01, gen throughput (token/s): 59.53, #queue-req: 0, 
[2025-03-18 06:25:18] INFO:     127.0.0.1:38536 - "POST /v1/chat/completions HTTP/1.1" 200 OK


Streaming mode is also supported.

In [4]:
stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

[2025-03-18 06:25:18] INFO:     127.0.0.1:38536 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-03-18 06:25:18 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0, 
It looks like you're preparing for a test or quiz. If you'd like to

 simulate a test or[2025-03-18 06:25:19 TP0] Decode batch. #running-req: 1, #token: 62, token usage: 0.00, gen throughput (token/s): 81.75, #queue-req: 0, 
 need help studying for one, I can assist you with questions and answers on

 a wide range of topics. What subject or type of test are you preparing for?

## Completions

### Usage
Completions API is similar to Chat Completions API, but without the `messages` parameter or chat templates.

In [5]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

print_highlight(f"Response: {response}")

[2025-03-18 06:25:19 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-18 06:25:19 TP0] Decode batch. #running-req: 1, #token: 17, token usage: 0.00, gen throughput (token/s): 80.00, #queue-req: 0, 


[2025-03-18 06:25:20 TP0] Decode batch. #running-req: 1, #token: 57, token usage: 0.00, gen throughput (token/s): 107.01, #queue-req: 0, 
[2025-03-18 06:25:20] INFO:     127.0.0.1:38536 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [6]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

print_highlight(f"Response: {response}")

[2025-03-18 06:25:20 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-18 06:25:20 TP0] Decode batch. #running-req: 1, #token: 34, token usage: 0.00, gen throughput (token/s): 98.06, #queue-req: 0, 


[2025-03-18 06:25:21 TP0] Decode batch. #running-req: 1, #token: 74, token usage: 0.00, gen throughput (token/s): 70.02, #queue-req: 0, 


[2025-03-18 06:25:21 TP0] Decode batch. #running-req: 1, #token: 114, token usage: 0.01, gen throughput (token/s): 102.07, #queue-req: 0, 


[2025-03-18 06:25:21 TP0] Decode batch. #running-req: 1, #token: 154, token usage: 0.01, gen throughput (token/s): 104.87, #queue-req: 0, 
[2025-03-18 06:25:21] INFO:     127.0.0.1:38536 - "POST /v1/completions HTTP/1.1" 200 OK


## Structured Outputs (JSON, Regex, EBNF)

For OpenAI compatible structed outputs API, refer to [Structured Outputs](https://docs.sglang.ai/backend/structured_outputs.html#OpenAI-Compatible-API) for more details.


## Batches

Batches API for chat completions and completions are also supported. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [7]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Batch job created with ID: {batch_response.id}")

[2025-03-18 06:25:22] INFO:     127.0.0.1:46964 - "POST /v1/files HTTP/1.1" 200 OK
[2025-03-18 06:25:22] INFO:     127.0.0.1:46964 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-03-18 06:25:22 TP0] Prefill batch. #new-seq: 2, #new-token: 20, #cached-token: 60, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [8]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        print_highlight(f"Request {result['custom_id']}:")
        print_highlight(f"Response: {result['response']}")

    print_highlight("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    print_highlight(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        print_highlight(f"Errors: {batch_response.errors}")

[2025-03-18 06:25:22 TP0] Decode batch. #running-req: 1, #token: 73, token usage: 0.00, gen throughput (token/s): 81.51, #queue-req: 0, 


Batch job status: validating...trying again in 3 seconds...
[2025-03-18 06:25:25] INFO:     127.0.0.1:46964 - "GET /v1/batches/batch_99a5c38e-0602-4a6c-8f5d-193d07fc29c7 HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2025-03-18 06:25:25] INFO:     127.0.0.1:46964 - "GET /v1/files/backend_result_file-89834d3c-2086-4a80-b1ec-b5e086f0b1b3/content HTTP/1.1" 200 OK


[2025-03-18 06:25:25] INFO:     127.0.0.1:46964 - "DELETE /v1/files/backend_result_file-89834d3c-2086-4a80-b1ec-b5e086f0b1b3 HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [9]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(20):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 64,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    print_highlight(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    print_highlight(
        f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
    )

    time.sleep(3)

[2025-03-18 06:25:25] INFO:     127.0.0.1:46972 - "POST /v1/files HTTP/1.1" 200 OK
[2025-03-18 06:25:25] INFO:     127.0.0.1:46972 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-03-18 06:25:25 TP0] Prefill batch. #new-seq: 2, #new-token: 60, #cached-token: 50, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-18 06:25:25 TP0] Prefill batch. #new-seq: 18, #new-token: 540, #cached-token: 450, token usage: 0.00, #running-req: 2, #queue-req: 0, 


[2025-03-18 06:25:25 TP0] Decode batch. #running-req: 20, #token: 1105, token usage: 0.05, gen throughput (token/s): 154.81, #queue-req: 0, 


[2025-03-18 06:25:26 TP0] Decode batch. #running-req: 20, #token: 20, token usage: 0.00, gen throughput (token/s): 1775.52, #queue-req: 0, 


[2025-03-18 06:25:35] INFO:     127.0.0.1:38350 - "GET /v1/batches/batch_fedd1188-6b12-4cb6-86d0-d5ece9a2fcba HTTP/1.1" 200 OK


[2025-03-18 06:25:38] INFO:     127.0.0.1:38350 - "GET /v1/batches/batch_fedd1188-6b12-4cb6-86d0-d5ece9a2fcba HTTP/1.1" 200 OK


[2025-03-18 06:25:41] INFO:     127.0.0.1:38350 - "GET /v1/batches/batch_fedd1188-6b12-4cb6-86d0-d5ece9a2fcba HTTP/1.1" 200 OK


[2025-03-18 06:25:44] INFO:     127.0.0.1:38350 - "GET /v1/batches/batch_fedd1188-6b12-4cb6-86d0-d5ece9a2fcba HTTP/1.1" 200 OK


[2025-03-18 06:25:47] INFO:     127.0.0.1:38350 - "GET /v1/batches/batch_fedd1188-6b12-4cb6-86d0-d5ece9a2fcba HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [10]:
import json
import time
from openai import OpenAI
import os

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(5000):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 128,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    print_highlight(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        print_highlight(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    print_highlight("Batch job successfully cancelled")

except Exception as e:
    print_highlight(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            print_highlight("Successfully cleaned up input file")
        if os.path.exists(input_file_path):
            os.remove(input_file_path)
            print_highlight("Successfully deleted local batch_requests.jsonl file")
    except Exception as e:
        print_highlight(f"Error cleaning up: {e}")
        raise e

[2025-03-18 06:25:50] INFO:     127.0.0.1:39146 - "POST /v1/files HTTP/1.1" 200 OK
[2025-03-18 06:25:50] INFO:     127.0.0.1:39146 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-03-18 06:25:51 TP0] Prefill batch. #new-seq: 15, #new-token: 15, #cached-token: 810, token usage: 0.02, #running-req: 0, #queue-req: 0, 
[2025-03-18 06:25:51 TP0] Prefill batch. #new-seq: 118, #new-token: 3395, #cached-token: 3095, token usage: 0.03, #running-req: 15, #queue-req: 285, 


[2025-03-18 06:25:53 TP0] Decode batch. #running-req: 133, #token: 9335, token usage: 0.46, gen throughput (token/s): 193.34, #queue-req: 4867, 


[2025-03-18 06:25:53 TP0] Decode batch. #running-req: 133, #token: 14655, token usage: 0.72, gen throughput (token/s): 8879.60, #queue-req: 4867, 


[2025-03-18 06:25:54 TP0] Decode batch. #running-req: 133, #token: 19975, token usage: 0.98, gen throughput (token/s): 10771.58, #queue-req: 4867, 
[2025-03-18 06:25:54 TP0] Decode out of memory happened. #retracted_reqs: 15, #new_token_ratio: 0.5766 -> 1.0000
[2025-03-18 06:25:54 TP0] Prefill batch. #new-seq: 11, #new-token: 330, #cached-token: 275, token usage: 0.89, #running-req: 118, #queue-req: 4871, 
[2025-03-18 06:25:54 TP0] Prefill batch. #new-seq: 118, #new-token: 3540, #cached-token: 2950, token usage: 0.02, #running-req: 11, #queue-req: 4753, 


...

[2025-03-18 06:26:00 TP0] Decode batch. #running-req: 132, #token: 9843, token usage: 0.48, gen throughput (token/s): 11180.07, #queue-req: 4360, 


[2025-03-18 06:26:00] INFO:     127.0.0.1:37602 - "POST /v1/batches/batch_c996314b-ec3d-4324-a873-153c774dd7ca/cancel HTTP/1.1" 200 OK


[2025-03-18 06:26:00 TP0] Prefill batch. #new-seq: 15, #new-token: 2295, #cached-token: 375, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-18 06:26:03] INFO:     127.0.0.1:37602 - "GET /v1/batches/batch_c996314b-ec3d-4324-a873-153c774dd7ca HTTP/1.1" 200 OK


[2025-03-18 06:26:03] INFO:     127.0.0.1:37602 - "DELETE /v1/files/backend_input_file-b7ca822f-c3b7-46e1-a5c7-111b300708b5 HTTP/1.1" 200 OK


In [11]:
terminate_process(server_process)